# **Projet Deep Learning 2023**

### *OUM Anthony & RAKOTOSON Lauren*

Importation des bibliothèques

In [15]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image

Définition de l'appareil (device) sur le CPU

In [16]:
device = torch.device("cpu")

Définition des chemins vers les répertoires d'entraînement et de test, ainsi que des fichiers de liste des images correspondantes

In [17]:
# Entrâinement
train_directory = "train"
train_file_path = "train.txt"

# Test
test_directory = "test"
test_file_path = "test.txt"

Initialisation des listes pour stocker les données d'entraînement, de validation et de test.

In [18]:
# Test
test_data = []

# Entraînement
training_data = []

# Validation
validation_data = []

### Entraînement du model

Lecture du fichier "train.txt"

In [19]:
with open(train_file_path, "r") as file:
    for i, line in enumerate(file):
        if i < 72000:
            parts = line.strip().split("\t")

            # Vérifier que la ligne contient au moins 2 éléments
            if len(parts) >= 2:
                image_name, label = parts[0], int(parts[1])

                # Créer un chemin d'accès complet pour l'image
                image_path = os.path.join(train_directory, image_name)

                # Ajouter les données à la liste training_data
                training_data.append((image_path, label))

        elif 72000 <= i < 90000:
            parts = line.strip().split("\t")

            # Vérifier que la ligne contient au moins 2 éléments
            if len(parts) >= 2:
                image_name, label = parts[0], int(parts[1])

                # Créer un chemin d'accès complet pour l'image
                image_path = os.path.join(train_directory, image_name)

                # Ajouter les données à la liste validation_data
                validation_data.append((image_path, label))

Affichage de quelques exemples de données d'entraînement et de validation

In [20]:
# Données d'entraînement
print("Exemples de données de training_data :")
for i in range(10):
    if i < len(training_data):
        print(training_data[i])

# Données de validation
print("\nExemples de données de validation_data :")
for i in range(10):
    if i < len(validation_data):
        print(validation_data[i])

Exemples de données de training_data :
('train\\000000.jpg', -1)
('train\\000001.jpg', 1)
('train\\000002.jpg', -1)
('train\\000003.jpg', -1)
('train\\000004.jpg', 1)
('train\\000005.jpg', -1)
('train\\000006.jpg', -1)
('train\\000007.jpg', 1)
('train\\000008.jpg', -1)
('train\\000009.jpg', -1)

Exemples de données de validation_data :
('train\\072000.jpg', -1)
('train\\072001.jpg', 1)
('train\\072002.jpg', -1)
('train\\072003.jpg', -1)
('train\\072004.jpg', -1)
('train\\072005.jpg', -1)
('train\\072006.jpg', -1)
('train\\072007.jpg', -1)
('train\\072008.jpg', -1)
('train\\072009.jpg', -1)


Chargement et prétraitement d'image

In [21]:
def load_and_preprocess_image(image_path):
    image = Image.open(image_path)
    preprocess = transforms.Compose([
        transforms.Resize((80, 80)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    preprocessed_image = preprocess(image)
    return preprocessed_image

Définition du modèle de classification amélioré

In [22]:
class CustomClassifier(nn.Module):
    def __init__(self):
        super(CustomClassifier, self).__init__()

        # Couches de convolution
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)

        # Couche de pooling maximale
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Couches entièrement connectées
        self.fc1 = nn.Linear(256 * 10 * 10, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 1)

        # Couche de dropout pour prévenir le surajustement
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):

        # Couches de convolution avec activation ReLU et pooling maximal
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        # Aplatir la sortie
        x = x.view(-1, 256 * 10 * 10)

        # Couches entièrement connectées avec activation ReLU et dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        
        # Couche de sortie
        x = self.fc3(x)
        return x

Définition du DataLoader pour les données d'entraînement et de validation

In [23]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path, label = self.data[idx]
        
        # Charger et prétraiter l'image
        image = load_and_preprocess_image(image_path)
        return image, label

Préparation des données et configuration de l'apprentissag

In [24]:
train_dataset = CustomDataset(training_data)
validation_dataset = CustomDataset(validation_data)

Configuration des DataLoaders pour charger les données en lots

In [25]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)
validation_loader = DataLoader(validation_dataset, batch_size=64, shuffle=False, num_workers=0)

Initialisation du modèle CustomClassifier

In [26]:
model = CustomClassifier()

Configuration de la fonction de perte (MarginRankingLoss) et de l'optimiseur (Adam)

In [27]:
criterion = nn.MarginRankingLoss(margin=1.0)
optimizer = optim.Adam(model.parameters(), lr=0.001)

Entraînement du modèle amélioré

In [28]:
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for batch in train_loader:
        inputs, labels = batch
        optimizer.zero_grad()
        outputs = model(inputs)
        
        # Ajuster les étiquettes à -1 et 1
        adjusted_labels = 2 * labels - 1
        
        # Ajouter une dimension à la sortie pour s'assurer qu'elle a la même forme que les étiquettes
        outputs = outputs.view(-1, 1)
        
        # Perte de marge de classement
        loss = criterion(outputs, torch.zeros_like(outputs), adjusted_labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")

Epoch 1/3, Loss: 0.23620837217238214
Epoch 2/3, Loss: 0.18056491725477908
Epoch 3/3, Loss: 0.15887348105841212


### Évaluation du modèle sur les données de validation

Mettre le modèle en mode évaluation

In [29]:
model.eval()

CustomClassifier(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=25600, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

Initialisation des compteurs pour le calcul de la précision

In [30]:
correct_predictions = 0
total_samples = 0

Désactivation de calcul du gradient pendant l'évaluation

In [31]:
with torch.no_grad():

    # Parcourir les lots dans le DataLoader de validation
    for batch in validation_loader:

        # Charger les entrées et les étiquettes du lot
        inputs, labels = batch

        # Effectuer une inférence avec le modèle
        outputs = model(inputs)

        # Appliquer une fonction d'activation et convertir les prédictions en labels binaires
        predicted = (outputs >= 0.0).view(-1)
        
        # Mettre à jour les compteurs de précision
        total_samples += labels.size(0)
        correct_predictions += (predicted * 2 - 1 == labels).sum().item()

Calcule de la précision finale et l'afficher

In [32]:
accuracy = 100 * correct_predictions / total_samples
print(f"Précision sur les données de validation : {accuracy:.2f}%")

Précision sur les données de validation : 90.30%


### Tester le model

Création d'une liste pour stocker les prédictions

In [33]:
test_predictions = []

Chargement et prétraitement de chaque image de test et faire des prédictions

In [34]:
with open(test_file_path, "r") as file:
    for line in file:

        # Supprimer les sauts de ligne et espaces blancs
        image_name = line.strip()

        # Construire le chemin complet de l'image
        image_path = os.path.join(test_directory, image_name)

        # Charger et prétraiter l'image
        input_image = load_and_preprocess_image(image_path)
        input_image = input_image.unsqueeze(0)
        input_image = input_image.to(device)

        # Ajouter le chemin complet à la liste test_data
        test_data.append(image_path)
        
        # Faire une prédiction
        with torch.no_grad():
            output = model(input_image)
            predicted_label = 1 if output.item() >= 0.0 else -1
            test_predictions.append(predicted_label)

            # Maintenant, la variable 'predicted_label' contient la prédiction pour cette image
            print(f"Image {image_name}: Prédiction = {predicted_label}")

Image 000000.jpg: Prédiction = -1
Image 000001.jpg: Prédiction = -1
Image 000002.jpg: Prédiction = -1
Image 000003.jpg: Prédiction = -1
Image 000004.jpg: Prédiction = 1
Image 000005.jpg: Prédiction = -1
Image 000006.jpg: Prédiction = -1
Image 000007.jpg: Prédiction = -1
Image 000008.jpg: Prédiction = 1
Image 000009.jpg: Prédiction = -1
Image 000010.jpg: Prédiction = -1
Image 000011.jpg: Prédiction = 1
Image 000012.jpg: Prédiction = -1
Image 000013.jpg: Prédiction = -1
Image 000014.jpg: Prédiction = -1
Image 000015.jpg: Prédiction = -1
Image 000016.jpg: Prédiction = 1
Image 000017.jpg: Prédiction = -1
Image 000018.jpg: Prédiction = -1
Image 000019.jpg: Prédiction = -1
Image 000020.jpg: Prédiction = -1
Image 000021.jpg: Prédiction = -1
Image 000022.jpg: Prédiction = -1
Image 000023.jpg: Prédiction = -1
Image 000024.jpg: Prédiction = 1
Image 000025.jpg: Prédiction = 1
Image 000026.jpg: Prédiction = 1
Image 000027.jpg: Prédiction = 1
Image 000028.jpg: Prédiction = -1
Image 000029.jpg: Préd

Création de fichier pour enregistrer les noms des images avec les labels prédits

In [35]:
with open("predictions_OUM_RAKOTOSON.txt", "w") as file:
    for image_path, predicted_label in zip(test_data, test_predictions):

        # Récupérer le nom de l'image à partir du chemin complet
        image_name = os.path.basename(image_path)
        
        # Écrire le nom de l'image avec le label prédit dans le fichier
        file.write(f"{image_name}\t{predicted_label}\n")